In [1]:
from datetime import date
import requests
import pandas as pd
import time
pd.set_option('display.max_columns', None)


# Setup

In [2]:
institution_ror = catalog.load('params:openalex_extract_options.institution_ror')
author_filter = catalog.load('params:openalex_extract_options.author_filter')
institution_filter = catalog.load('params:openalex_extract_options.institution_filter')
work_filter = catalog.load('params:openalex_extract_options.work_filter')

#env = catalog.load('params:fetch_options.env')
env = 'dev'

print(f'institution_ror: {institution_ror}')
print(f'author_filter: {author_filter}')
print(f'institution_filter: {institution_filter}')
print(f'work_filter: {work_filter}')
print(f'env: {env}')


                    INFO     Loading data from params:openalex_extract_options.institution_ror  ]8;id=544560;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=443480;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:openalex_extract_options.author_filter    ]8;id=129989;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=84181;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from                                                  ]8;id=900959;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=495444;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             params:openalex_extract_options.institution_filter                                    
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:openalex_extract_options.work_filter      ]8;id=692114;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=1532;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

institution_ror: https://ror.org/02s7sax82
author_filter: affiliations.institution.ror
institution_filter: ror
work_filter: institutions.ror
env: dev


# Node

In [3]:
def openalex_extract(institution_ror: str, filter_field: str, entity: str = 'institutions', env: str = 'dev', cleaner=None):
    """
    Fetch data from OpenAlex API for a given entity and institution ROR.

    Args:
        entity (str): 'authors', 'institutions', 'works', etc.
        institution_ror (str): ROR id of the institution.
        env (str): 'dev' or 'prod'.
        filter_field (str): the filter key to use (e.g. 'affiliations.institution.ror').
        cleaner (callable): function to clean DataFrame columns, optional.

    Returns:
        pd.DataFrame: full concatenated results
        pd.DataFrame: head(1000) sample
    """
    session = requests.Session()
    base_url = f"https://api.openalex.org/{entity}?filter={filter_field}:{{}}&cursor={{}}&per-page=200"
    cursor = '*'
    iteration_limit = 5
    iteration_count = 0
    all_dataframes = []

    while True:
        url = base_url.format(institution_ror, cursor)
        print(f'Iteration count: {iteration_count}')
        print(f'GET {url}')

        try:
            response = session.get(url, timeout=10)
            response.raise_for_status()
            api_response = response.json()
        except requests.RequestException as e:
            print(f"Error en la solicitud: {e}")
            break
        except ValueError:
            print("Error al decodificar JSON.")
            break

        if 'results' not in api_response or not api_response['results']:
            print("No hay más datos disponibles.")
            break

        df_tmp = pd.DataFrame.from_dict(api_response['results'])

        columns_to_drop = {"abstract_inverted_index", "abstract_inverted_index_v3"}
        df_tmp = df_tmp.drop(columns=columns_to_drop.intersection(df_tmp.columns))

        all_dataframes.append(df_tmp)

        # update cursor
        cursor = api_response.get('meta', {}).get('next_cursor')
        if not cursor:
            break

        iteration_count += 1
        if env == 'dev' and iteration_count >= iteration_limit:
            break

        time.sleep(1)

    df = pd.concat(all_dataframes, ignore_index=True) if all_dataframes else pd.DataFrame()


    df['extract_datetime'] = date.today()

    return df, df.head(1000)


# Results

In [4]:
df_institution, df_institution_dev = openalex_extract(institution_ror, institution_filter)

Iteration count: 0
GET https://api.openalex.org/institutions?filter=ror:https://ror.org/02s7sax82&cursor=*&per-page=200
Iteration count: 1
GET https://api.openalex.org/institutions?filter=ror:https://ror.org/02s7sax82&cursor=IlszNTEwLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvSTQyMTAxMjI5NzAnXSI=&per-page=200
No hay más datos disponibles.


In [5]:
df_institution_dev

,id,ror,display_name,country_code,type,type_id,lineage,homepage_url,image_url,image_thumbnail_url,display_name_acronyms,display_name_alternatives,repositories,works_count,cited_by_count,summary_stats,ids,geo,international,associated_institutions,counts_by_year,roles,topics,topic_share,x_concepts,is_super_system,works_api_url,updated_date,created_date,extract_datetime
0,https://openalex.org/I4210122970,https://ror.org/02s7sax82,Comisión de Investigaciones Científicas,AR,government,https://openalex.org/institution-types/government,[https://openalex.org/I4210122970],https://www.cic.gba.gob.ar,None,None,[CIC],[],[],3510,56910,"{'2yr_mean_citedness': 1.4695652173913043, 'h_...",{'openalex': 'https://openalex.org/I4210122970...,"{'city': 'La Plata', 'geonames_city_id': '3432...",{'display_name': {'en': 'Comisión de Investiga...,"[{'id': 'https://openalex.org/I4411590623', 'r...","[{'year': 2025, 'works_count': 78, 'oa_works_c...","[{'role': 'funder', 'id': 'https://openalex.or...","[{'id': 'https://openalex.org/T10825', 'displa...","[{'id': 'https://openalex.org/T10650', 'displa...","[{'id': 'https://openalex.org/C86803240', 'wik...",False,https://api.openalex.org/works?filter=institut...,2025-09-25T10:26:50.738232,2022-02-02,2025-09-28


In [6]:
df_author, df_author_dev = openalex_extract(institution_ror, author_filter, 'authors')

Iteration count: 0
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=*&per-page=200
Iteration count: 1
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=IlsxMDQsICdodHRwczovL29wZW5hbGV4Lm9yZy9BNTAxMTEwMDU0MCddIg==&per-page=200
Iteration count: 2
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=Ils2NywgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL0E1MDAwOTU1ODg3J10i&per-page=200
Iteration count: 3
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=Ils0NCwgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL0E1MDQ2NzI4Njg2J10i&per-page=200
Iteration count: 4
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=IlszMiwgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL0E1MDYwMjY4NDAwJ10i&per-page=200


In [7]:
df_author_dev

,id,orcid,display_name,display_name_alternatives,works_count,cited_by_count,summary_stats,ids,affiliations,last_known_institutions,topics,topic_share,x_concepts,counts_by_year,works_api_url,updated_date,created_date,extract_datetime
0,https://openalex.org/A5003447605,https://orcid.org/0000-0002-5260-1807,Gustavo E. Romero,"[Gustavo Adolfo Sierra Romero, G. E Romero, Gu...",772,9844,"{'2yr_mean_citedness': 0.9038461538461539, 'h_...",{'openalex': 'https://openalex.org/A5003447605...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I4210134895', 'r...","[{'id': 'https://openalex.org/T10818', 'displa...","[{'id': 'https://openalex.org/T10818', 'displa...","[{'id': 'https://openalex.org/C121332964', 'wi...","[{'year': 2025, 'works_count': 8, 'oa_works_co...",https://api.openalex.org/works?filter=author.i...,2025-09-19T09:50:53.045401,2023-07-21,2025-09-28
1,https://openalex.org/A5053874532,None,Guillermo Soberón,"[Guillermo R Soberon, Guillermo Soberon, G. So...",630,1146,"{'2yr_mean_citedness': 0.0, 'h_index': 14, 'i1...",{'openalex': 'https://openalex.org/A5053874532'},[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I4210143748', 'r...","[{'id': 'https://openalex.org/T13464', 'displa...","[{'id': 'https://openalex.org/T13403', 'displa...","[{'id': 'https://openalex.org/C138885662', 'wi...","[{'year': 2025, 'works_count': 0, 'oa_works_co...",https://api.openalex.org/works?filter=author.i...,2025-09-20T01:52:26.776348,2023-07-21,2025-09-28
2,https://openalex.org/A5112709549,https://orcid.org/0000-0002-6058-0399,Hugo Luis López,"[Hugo L. López, Hugo Luis López, Hugo L. Lopez...",484,3447,"{'2yr_mean_citedness': 1.3333333333333333, 'h_...",{'openalex': 'https://openalex.org/A5112709549...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I96355252', 'ror...","[{'id': 'https://openalex.org/T13589', 'displa...","[{'id': 'https://openalex.org/T13589', 'displa...","[{'id': 'https://openalex.org/C142362112', 'wi...","[{'year': 2025, 'works_count': 2, 'oa_works_co...",https://api.openalex.org/works?filter=author.i...,2025-09-25T11:33:31.188781,2024-10-15,2025-09-28
3,https://openalex.org/A5019186384,https://orcid.org/0000-0001-6247-0428,Guillermo Ricardo Simari,"[Guillermo Ricardo Simari, G. Simari, Guillerm...",467,6073,"{'2yr_mean_citedness': 1.0, 'h_index': 33, 'i1...",{'openalex': 'https://openalex.org/A5019186384...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I56858762', 'ror...","[{'id': 'https://openalex.org/T10456', 'displa...","[{'id': 'https://openalex.org/T10456', 'displa...","[{'id': 'https://openalex.org/C138885662', 'wi...","[{'year': 2025, 'works_count': 0, 'oa_works_co...",https://api.openalex.org/works?filter=author.i...,2025-09-21T06:55:32.687562,2023-07-21,2025-09-28
4,https://openalex.org/A5062756128,https://orcid.org/0000-0002-1030-4492,Peter Vogt,"[Peter Vogt, Peter R. Vogt, P. Vogt, V. Peter,...",421,16276,"{'2yr_mean_citedness': 5.636363636363637, 'h_i...",{'openalex': 'https://openalex.org/A5062756128...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I1288214837', 'r...","[{'id': 'https://openalex.org/T10001', 'displa...","[{'id': 'https://openalex.org/T14323', 'displa...","[{'id': 'https://openalex.org/C86803240', 'wik...","[{'year': 2025, 'works_count': 3, 'oa_works_co...",https://api.openalex.org/works?filter=author.i...,2025-09-21T09:10:45.305208,2023-07-21,2025-09-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://openalex.org/A5005317223,https://orcid.org/0000-0003-3944-5093,Carolina Griselda Luchetti,"[Carolina Luchetti, C. Luchetti, Carolina Gris...",23,406,"{'2yr_mean_citedness': 2.0, 'h_index': 10, 'i1...",{'openalex': 'https://openalex.org/A5005317223...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I24354313', 'ror...","[{'id': 'https://openalex.org/T10364', 'displ

In [8]:
df_work, df_work_dev = openalex_extract(institution_ror, work_filter, 'works')

Iteration count: 0
GET https://api.openalex.org/works?filter=institutions.ror:https://ror.org/02s7sax82&cursor=*&per-page=200
Iteration count: 1
GET https://api.openalex.org/works?filter=institutions.ror:https://ror.org/02s7sax82&cursor=Ils5Ni4wLCA2NSwgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL1cxOTY3NDM3MzQ2J10i&per-page=200
Iteration count: 2
GET https://api.openalex.org/works?filter=institutions.ror:https://ror.org/02s7sax82&cursor=Ils5NC4wLCAzNSwgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL1cyMDEyNTY1MzA3J10i&per-page=200
Iteration count: 3
GET https://api.openalex.org/works?filter=institutions.ror:https://ror.org/02s7sax82&cursor=Ils5Mi4wLCAyNSwgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL1cyMDAwNTE2NjA5J10i&per-page=200
Iteration count: 4
GET https://api.openalex.org/works?filter=institutions.ror:https://ror.org/02s7sax82&cursor=Ils5MC4wLCAxOCwgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL1cxNTEyNDczMjg4J10i&per-page=200


In [9]:
df_work_dev

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,type_crossref,indexed_in,open_access,authorships,institution_assertions,countries_distinct_count,institutions_distinct_count,corresponding_author_ids,corresponding_institution_ids,apc_list,apc_paid,fwci,has_fulltext,fulltext_origin,cited_by_count,citation_normalized_percentile,cited_by_percentile_year,biblio,is_retracted,is_paratext,primary_topic,topics,keywords,concepts,mesh,locations_count,locations,best_oa_location,sustainable_development_goals,grants,datasets,versions,referenced_works_count,referenced_works,related_works,cited_by_api_url,counts_by_year,updated_date,created_date,extract_datetime
0,https://openalex.org/W2089852843,https://doi.org/10.1016/s0378-4371(98)00437-3,The role of constraints within generalized non...,The role of constraints within generalized non...,1998,1998-12-01,{'openalex': 'https://openalex.org/W2089852843...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,journal-article,[crossref],"{'is_oa': False, 'oa_status': 'closed', 'oa_ur...","[{'author_position': 'first', 'author': {'id':...",[],3,6,[https://openalex.org/A5051451815],[https://openalex.org/I4210125245],"{'value': 3320, 'currency': 'USD', 'value_usd'...",None,30.138,True,ngrams,1366,"{'value': 0.999703, 'is_in_top_1_percent': Tru...","{'min': 99, 'max': 100}","{'volume': '261', 'issue': '3-4', 'first_page'...",False,False,"{'id': 'https://openalex.org/T12261', 'display...","[{'id': 'https://openalex.org/T12261', 'displa...",[{'id': 'https://openalex.org/keywords/hamilto...,"[{'id': 'https://openalex.org/C40934718', 'wik...",[],1,"[{'is_oa': False, 'landing_page_url': 'https:/...",None,"[{'score': 0.82, 'display_name': 'Affordable a...",[],[],[],74,"[https://openalex.org/W1553236419, https://ope...","[https://openalex.org/W620348747, https://open...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2025, 'cited_by_count': 25}, {'year'...",2025-09-21T03:36:10.930753,2016-06-24,2025-09-28
1,https://openalex.org/W3037375387,https://doi.org/10.1016/j.envint.2020.105876,Short-term exposure to particulate matter (PM1...,Short-term exposure to particulate matter (PM1...,2020,2020-06-23,{'openalex': 'https://openalex.org/W3037375387...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",review,journal-article,"[crossref, doaj, pubmed]","{'is_oa': True, 'oa_status': 'gold', 'oa_url':...","[{'author_position': 'first', 'author': {'id':...",[],1,5,[https://openalex.org/A5069680875],"[https://openalex.org/I151201029, https://open...","{'value': 3500, 'currency': 'USD', 'value_usd'...","{'value': 3500, 'currency': 'USD', 'value_usd'...",8.608,True,pdf,621,"{'value': 0.997775, 'is_in_top_1_percent': Tru...","{'min': 99, 'max': 100}","{'volume': '142', 'issue': None, 'first_page':...",False,False,"{'id': 'https://openalex.org/T10190', 'display...","[{'id': 'https://openalex.org/T10190', 'displa...",[{'id': 'https://openalex.org/keywords/nitroge...,"[{'id': 'https://openalex.org/C2780723490', 'w...","[{'descriptor_ui': 'D000393', 'descriptor_name...",4,"[{'is_oa': True, 'landing_page_url': 'https://...","{'is_oa': True, 'landing_page_url': 'https://d...","[{'score': 0.81, 'display_name': 'Good health ...",[{'funder': 'https://openalex.org/F4320307856'...,[],[],85,"[https://openalex.org/W1525116145, https://ope...","[https://openalex.org/W4399338999, https://ope...",https://api.openalex.org/works?filter=cites:W3...,"[{'year': 2025, 'cited_by_count': 93}, {'year'...",2025-09-23T04:37:08.866086,2020-07-02,2025-09-28
2,https://openalex.org/W2032143374,https://doi.org/10.1016/s0023-6438(03)00088-4,Antimicrobial activity of essential oils on th...,Antimicrobial activity of essential oils on th...,2003,2003-11-01,{'openalex': 'https://openalex.org/W2032143374...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,journal-article,[crossref],"{'is_oa': False, 'oa_status': 'closed', 'oa_ur...","[{'author_position': 'first', 'autho